In [1]:
import pickle, random, os
import numpy as np
#import skimage.data as imd
import matplotlib.pyplot as plt
import tensorflow as tf
#from tensorflow.python.framework import ops
#from skimage import transform
#from skimage.color import rgb2gray

## Cogemos los datos

In [2]:
infile = open('data/data8.pickle','rb')
new_dict = pickle.load(infile)
y_train = new_dict['y_train']
x_train = new_dict['x_train']
y_test = new_dict['y_test']
x_test = new_dict['x_test']
new_dict['x_validation'];

FileNotFoundError: [Errno 2] No such file or directory: 'data/data8.pickle'

In [ ]:
batch_size = 100
learning_rate = 0.005
evaluation_size = 500
image_width = x_train[0].shape[0]
image_height = x_train[0].shape[1]
target_size = max(labels_train)+1
num_chanels = 1
generations = 2500
eval_every = 5
conv1_features = 25
conv2_features = 50
max_pool_size1 = 2
max_pool_size2 = 2
full_connected_size1 = 120

In [ ]:
x_input_shape = (batch_size, image_width, image_height, num_chanels)
x_input = tf.placeholder(tf.float32, shape = x_input_shape)
y_target = tf.placeholder(tf.int32, shape=(batch_size))

eval_input_shape = (None, image_width, image_height, num_chanels)
eval_input = tf.placeholder(tf.float32, shape=eval_input_shape)
eval_target = tf.placeholder(tf.float32, shape = (evaluation_size))

In [ ]:
conv1_weight = tf.Variable(tf.truncated_normal([4,4,num_chanels, conv1_features], stddev=0.1, dtype=tf.float32))
conv1_bias = tf.Variable(tf.zeros([conv1_features], dtype=tf.float32))
                    
conv2_weight = tf.Variable(tf.truncated_normal([4,4,conv1_features, conv2_features], stddev=0.1, dtype=tf.float32))
conv2_bias = tf.Variable(tf.zeros([conv2_features], dtype=tf.float32))

In [ ]:
resulting_width = image_width // (max_pool_size1*max_pool_size2)
resulting_height = image_height // (max_pool_size1 * max_pool_size2)

full1_input_size = resulting_width*resulting_height*conv2_features
full1_weight = tf.Variable(tf.truncated_normal([full1_input_size, full_connected_size1], stddev=0.1, dtype=tf.float32))
full1_bias = tf.Variable(tf.truncated_normal([full_connected_size1], stddev=0.1, dtype = tf.float32))

full2_weight = tf.Variable(tf.truncated_normal([full_connected_size1, full_connected_size1], stddev=0.1, dtype=tf.float32))
full2_bias = tf.Variable(tf.truncated_normal([full_connected_size1], stddev=0.1, dtype=tf.float32))

full3_weight = tf.Variable(tf.truncated_normal([full_connected_size1, target_size], stddev=0.1, dtype=tf.float32))
full3_bias = tf.Variable(tf.truncated_normal([target_size], stddev=0.1, dtype=tf.float32))

In [ ]:
def my_advanced_conv_neural_net(input_data):
    # First Layer: Conv+ReLU+Maxpool+BatchNormalization
    conv1 = tf.nn.conv2d(input_data, conv1_weight, strides=[1,1,1,1], padding="SAME")
    relu1 = tf.nn.relu(tf.nn.bias_add(conv1, conv1_bias))
    max_pool1 = tf.nn.max_pool(relu1, ksize=[1,max_pool_size1, max_pool_size1,1], 
                               strides=[1, max_pool_size1, max_pool_size1,1], padding="SAME")
    norm_1 = tf.nn.lrn(max_pool1, depth_radius=5, bias=2.0, alpha=1e-3, beta=0.75)
    # Second layer: Conv+ReLU+Maxpool+BatchNormalization
    conv2 = tf.nn.conv2d(norm_1, conv2_weight, strides=[1,1,1,1], padding="SAME")
    relu2 = tf.nn.relu(tf.nn.bias_add(conv2, conv2_bias))
    max_pool2 = tf.nn.max_pool(relu2, ksize=[1,max_pool_size2, max_pool_size2,1], 
                               strides=[1, max_pool_size2, max_pool_size2,1], padding="SAME")
    norm_2 = tf.nn.lrn(max_pool2, depth_radius=5, bias=2.0, alpha=1e-3, beta=0.75)
    # Flattening
    flat_output = tf.contrib.layers.flatten(norm_2)
    # Third layer: Fully connected
    fully_connected_1 = tf.nn.relu(tf.add(tf.matmul(flat_output, full1_weight), full1_bias))
    fully_connected_1 = tf.nn.dropout(fully_connected_1, 0.87)
    # Fourth layer: Fully connected
    fully_connected_2 = tf.nn.relu(tf.add(tf.matmul(fully_connected_1, full2_weight), full2_bias))
    fully_connected_2 = tf.nn.dropout(fully_connected_2, 0.87)
    # Fith layer: Fully connected
    fully_connected_3 = tf.add(tf.matmul(fully_connected_2, full3_weight), full3_bias)
    return fully_connected_3

In [ ]:
model_ouput = my_advanced_conv_neural_net(x_input)
test_model_output = my_advanced_conv_neural_net(eval_input)

In [ ]:
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model_ouput, labels = y_target))

In [ ]:
prediction = tf.nn.softmax(model_ouput)
test_prediction = tf.nn.softmax(test_model_output)

In [ ]:
my_optim = tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum=0.9)
train_step = my_optim.minimize(loss)

In [ ]:
init = tf.global_variables_initializer()
sess.run(init)

In [ ]:
train_loss = []
train_acc = []
test_acc = []
i_vals = []
for i in range(generations):
    rand_idx = np.random.choice(len(images_train_28_gray), size = batch_size)
    rand_x = images_train_28_gray[rand_idx]
    rand_x = np.expand_dims(rand_x, 3)
    rand_y = labels_train[rand_idx]
    train_dict = {x_input:rand_x, y_target:rand_y}
    sess.run(train_step, feed_dict=train_dict)
    temp_train_loss, temp_train_preds = sess.run([loss, prediction], feed_dict=train_dict)
    temp_train_acc = get_accuracy(temp_train_preds, rand_y)
    if(i+1) % eval_every == 0:
        rand_idx_eval = np.random.choice(len(images_test_28_gray), size = evaluation_size)
        rand_x_eval = images_test_28_gray[rand_idx_eval]
        rand_x_eval = np.expand_dims(rand_x_eval, 3)
        rand_y_eval = labels_test[rand_idx_eval]
        test_dict = {eval_input:rand_x_eval, eval_target:rand_y_eval}
        temp_test_preds = sess.run( test_prediction, feed_dict=test_dict)
        temp_test_acc = get_accuracy(temp_test_preds, rand_y_eval)
        
        i_vals.append(i+1)
        train_loss.append(temp_train_loss)
        train_acc.append(temp_train_acc)
        test_acc.append(temp_test_acc)
 
        acc_and_loss = [(i+1),temp_train_loss, temp_train_acc, temp_test_acc]
        acc_and_loss = [np.round(x,3) for x in acc_and_loss]
        print("Iteration {}. Train Loss: {:.3f}. Train Acc: {:.3f}. Test Acc: {:.3f}".format(*acc_and_loss))

## CNN 2